# Snelle inleiding threads


## Multitasking
Op moderne hardware en besturingssystemen kunnen programma's gelijktijdig en onafhankelijk van elkaar worden uitgevoerd. Dit wordt **multitasking** genoemd.

Het maakt daarbij niet uit of er daadwerkelijk fysiek meerdere processoren aanwezig zijn, meerdere processorkernen of dat het besturingssysteem er voor zorgt dat programma's om en om een klein stukje tijd krijgen toegewezen (wat zo vaak gebeurt dat het lijkt alsof programma's gelijktijdig worden uitgevoerd). Uiteraard bieden meerdere processoren en processorkernen een hogere prestatie dan één fysieke processor.

## Multithreading
Multithreading gaat nog een stapje verder: Niet alleen programma's worden gelijktijdig uitgevoerd, maar binnen een programma kunnen bepaalde delen gelijktijdig en onafhankelijk van elkaar worden uitgevoerd. Dit wordt **multithreading** genoemd.

Een programma wordt in dat geval opgesplists in **threads**.

````{note} Multithreading de basis
Multithreading is een groot onderwerp, waar complete boeken over zijn geschreven. Het gelijktijdig uitvoeren van delen van een programmma zorgt voor veel complexiteit. Bijvoorbeeld: Wat als verschillende delen gelijktijdig een object veranderen? In deze inleiding wordt alleen de basis besproken.
````


## Een voorbeeld

De klasse Counter telt van 1 tot en met een maximale waarde. Tussen de tellen wordt een aantal milliseconden gewacht.

```java
public class Counter {

    private String name;
    private int maxvalue;
    private long pause;

    public Counter(String name, int maxvalue, long pause) {
        this.name = name;
        this.maxvalue = maxvalue;
        this.pause = pause;
    }
    
    public void run() {
        int value=1;
        do {
            System.out.println(name+" - "+value);
            value++;
            try {
                Thread.sleep(pause);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        } while(value<=maxvalue);
    }
}
```

Via de constructor worden de instantievariabelen **name**, **maxvalue** en **pause** geïnitialiseerd.

De methode **run** voert het daadwerkelijke tellen uit.

De methode **Thread.sleep** zorgt voor een pauze. Omdat deze mogelijk de checked exception **InterruptedException** gooit, is een *try..catch-block* noodzakelijk.

Vanuit een main-methode is het eenvoudig om een aantal tellers te maken en uit te voeren:
```code
    Counter c1 = new Counter("Counter 1", 20, 200);
    Counter c2 = new Counter("Counter 2", 10, 700);
    Counter c3 = new Counter("Counter 3", 10, 340);
    c1.run();
    c2.run();
    c3.run();
```

De tellers worden <u>achter elkaar</u> uitgevoerd. Eerst teller 1, dan teller 2 en tot slot teller 3. Teller 1 gaat duidelijk sneller dan teller 2. Maar het gebeurt achter elkaar, in een volledig voorspelbare volgorde.

## Multithreading

Een Java-programma bestaat uit minimaal één thread die de main-methode start. Daarnaast kunnen meerdere threads worden gemaakt.

Dit gebeurt als volgt:
- Maak een instantie van een klasse die de interface **Runnable** implementeert. De methode **run** bevat de code die door de thread moet worden uitgevoerd.
- Maak een instantie van klasse *Thread*, geef de instantie van de thread die **Runnable** imlementeert mee aan de constructor.
- Roep de methode **start** aan van het thread-object om de thread te starten.


## De klasse Counter in een thread

Voordat de Counter met een thread kan worden gebruikt, dient een kleine aanpassing te worden gemaakt: Counter moet de interface **Runnable** implementeren:
```Java
public class Counter implements Runnable {

    private String name;
    private int maxvalue;
    private long pause;

    public Counter(String name, int maxvalue, long pause) {
        this.name = name;
        this.maxvalue = maxvalue;
        this.pause = pause;
    }

    @Override
    public void run() {
        int value=1;
        do {
            System.out.println(name+" - "+value);
            value++;
            try {
                Thread.sleep(pause);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        } while(value<=maxvalue);
    }
    
}
```

De interface Runnable bevat één methode:
```Java
void run()
```

Vervolgens kan de Counter met een thread worden uitgevoerd:
```Java
    Counter c1 = new Counter("Counter 1", 20, 200);
    Counter c2 = new Counter("Counter 2", 10, 700);
    Counter c3 = new Counter("Counter 3", 10, 340);
    Thread t1=new Thread(c1);
    Thread t2=new Thread(c2);
    Thread t3=new Thread(c3);
    t1.start();
    t2.start();
    t3.start();
```

De tellers vinden hiermee gelijktijdig plaats. De meldingen van de tellers komen door elkaar op de console. De volgorde is niet meer geheel voorspelbaar.

Let op! Veel gemaakte beginnersfout bij het werken met threads is dat in plaats van de methode **start** de methode **run** wordt aangeroepen. De methode **run** voert de inhoud van de methode **run** uit, maar niet in de betreffende thread zodat er geen sprake is van multithreading.



## Een modernere aanpak

Het rechtsteeks gebruik maken van de klasse **Thread** is de traditionele manier om threads te gebruiken in Java.

Een modernere aanpak is het gebruik van threadpools:
```java
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
import java.util.concurrent.TimeUnit;

    Counter c1 = new Counter("Counter 1", 20, 200);
    Counter c2 = new Counter("Counter 2", 10, 700);
    Counter c3 = new Counter("Counter 3", 10, 340);
    // Threadpool maken
    ExecutorService executor = Executors.newFixedThreadPool(3);
    // Runnables toevoegen aan de threadpool (threads worden automatisch gemaakt en gestart)
    executor.submit(c1);
    executor.submit(c2);
    executor.submit(c3);
    // Executor service afsluiten (uitvoering de threads gaat onverminderd door)
    executor.shutdown();
    // Wachten tot alle threads klaar zijn (met timeout van 10 seconden)
    try {
        executor.awaitTermination(10, TimeUnit.SECONDS);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
```